In [1]:
import torch

torch.__version__, torch.cuda.is_available()

/home/juan/Desktop/CUDA100/.venv/lib/python3.12/site-packages/torch/_subclasses/functional_tensor.py:279: UserWarning: Failed to initialize NumPy: No module named 'numpy' (Triggered internally at /pytorch/torch/csrc/utils/tensor_numpy.cpp:84.)
  cpu = _conversion_method_template(device=torch.device("cpu"))


('2.9.0+cu128', True)

In [5]:
a = torch.tensor([1.0, 2.0, 3.0, 4.0])
b = torch.tensor([5.0, 6.0, 7.0, 8.0])

c = a + b

assert (c == torch.tensor([6.0, 8.0, 10.0, 12.0])).all()

In [6]:
c

tensor([ 6.,  8., 10., 12.])